<a href="https://colab.research.google.com/github/FranciscoOcampoPredictiva/azureml_course/blob/main/Training_Script_for_the_explainer_Job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from azureml.core import Run

# Get the run context
new_run = Run.get_context()

# Access the workspace
ws = new_run.experiment.workspace

In [ ]:
# Get parameters
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str)
args = parser.parse_args()

In [ ]:
import pandas as pd

# Access the dataset
df = new_run.input_datasets['raw_data'].to_pandas_dataframe()

# Drop columns
df = df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

# Dummy variables
df = pd.get_dummies(df, drop_first=True)

# Define X and Y
X = df.drop(columns='Exited')
Y = df['Exited']

# Split Dataset into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0, stratify=Y)

# Train the ML model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(random_state=0)
trained_ml_model = classifier.fit(X_train, Y_train)

#  Predict the results
y_pred = classifier.predict(X_test)

# Model evaluation
from sklearn.metrics import confusion_matrix
cm    = confusion_matrix(Y_test, y_pred)
score = classifier.score(X_test, Y_test)

In [ ]:
# Logging the primary metric
new_run.log("accuracy", score)

In [ ]:
# Get explanation
from interpret_community.tabular_explainer import TabularExplainer

# Define variables
features = list(X.columns)
classes = ['Exited', 'Not Exited']

# tabular explainer object
tab_explainer = TabularExplainer(model=trained_ml_model,
                                 initialization_examples=X_train,
                                 features=features,
                                 classes=classes)

# explainations
explaination = tab_explainer.explain_global(X_train)

In [ ]:
# Upload explanations to workspace
from azureml.interpret import ExplanationClient

# Create explanation client
explanation_client = ExplanationClient.from_run(new_run)

# Upload the explanations
explanation_client.upload_model_explanation(explaination, comment="Explanations")

# Complete the run
new_run.complete()